# 构建员工排班

In [1]:
import numpy as np
from itertools import combinations
import random
import pandas as pd

In [2]:
# 休息排班
def generate_01_strings(ones, zeros, total_length):
    # 生成所有可能的1的位置组合
    all_positions = list(range(total_length))
    valid_combinations = combinations(all_positions, ones)
    binary_strings = []
    # 生成所有符合条件的二进制串
    for combo in valid_combinations:
        strings = ''.join('1' if i in combo else '0' for i in range(total_length))
        binary_strings.append(strings)

    # 要删除的元素列表
    elements_to_remove = ['1010111', '0101111', '1111010', '1110101', '1101011', '1011101', '1011011', '1101101','0111101','1011110']
    binary_strings = [s for s in binary_strings if s not in elements_to_remove]

    # 添加特定值的元素
    binary_strings.extend(['0011111', '1100111','1110011', '1111100','1001111', '1111001','1001111', '1111001','0011111','1111100'])

    random.shuffle(binary_strings)
    return binary_strings

In [3]:
strings_01 = generate_01_strings(5,2,7)

In [4]:
def get_worker_schedule(strings_01):
    work_list = strings_01*2
    random.shuffle(work_list)
    worker_schedule = []
    for s in work_list:
        split_list = [ int(s[i]) for i in range(len(s))]
        worker_schedule.append(split_list)
    worker_schedule = np.array(worker_schedule).T
    return worker_schedule

In [5]:
def get_worker_schedule_123(worker_schedule):
    matrix = worker_schedule.copy()
    yesterday = []
    # 对每行进行操作
    for i, row in enumerate(matrix):
    
        # 找出1的位置
        ones_positions = np.where(row == 1)[0]
        # 找出前一天3的位置
        yesterday_three_positions = np.where(row == 3)[0]

        # 选择10个前一天不为3的位置替换为1
        remaining_ones = list(set(ones_positions) - set(yesterday_three_positions))
        to_replace_one = np.random.choice(remaining_ones, size=10, replace=False)
        matrix[i, to_replace_one] = 1

        # 选择10个位置为2
        remaining_twos = list(set(ones_positions) - set(to_replace_one))
        to_replace_two = np.random.choice(remaining_twos, size=10, replace=False)
        matrix[i, to_replace_two] = 2

        # 选择10个位置为3
        remaining_threes = list(set(ones_positions) - set(to_replace_one) - set(to_replace_two))
        to_replace_three = np.random.choice(remaining_threes, size=10, replace=False)
        matrix[i, to_replace_three] = 3

        yesterday = matrix[i]
    return matrix

In [ ]:
np.random.seed(0)  # 为了可重现的结果

while True:
    strings_01 = generate_01_strings(5,2,7)
    worker_schedule = get_worker_schedule(strings_01)
    matrix = get_worker_schedule_123(worker_schedule)
    
    # 统计每列1, 2, 3的个数
    counts = np.array([[np.sum(matrix[:, j] == i) for i in range(1, 4)] for j in range(matrix.shape[1])])
    
    # 找出不符合条件的
    rows_with_45 = np.any((counts>3) , axis=1)
    rows_with_0 = np.any((counts == 0) , axis=1)

    # 如果符合条件，退出循环
    if (len(counts[rows_with_45]) == 0) & (len(counts[rows_with_0]) < 5) :
        break

array([[2, 3, 2, 3, 0, 2, 0, 2, 1, 0, 2, 0, 3, 0, 0, 3, 1, 1, 2, 3, 1, 3,
        0, 2, 0, 3, 0, 1, 1, 3, 3, 1, 0, 3, 0, 1, 2, 1, 1, 0, 2, 2],
       [0, 0, 3, 0, 2, 3, 2, 1, 2, 0, 1, 0, 3, 0, 0, 3, 2, 0, 1, 2, 1, 1,
        3, 1, 0, 1, 0, 1, 2, 0, 0, 3, 2, 1, 3, 3, 1, 3, 3, 2, 2, 2],
       [0, 0, 1, 0, 1, 3, 2, 1, 0, 2, 2, 2, 1, 2, 2, 0, 3, 0, 3, 1, 3, 0,
        1, 2, 3, 1, 3, 1, 1, 0, 0, 1, 3, 3, 2, 2, 3, 0, 2, 3, 0, 0],
       [3, 1, 0, 1, 2, 0, 3, 3, 0, 3, 2, 2, 0, 1, 3, 0, 1, 1, 0, 0, 2, 0,
        0, 0, 2, 2, 3, 3, 2, 3, 1, 3, 2, 2, 1, 2, 1, 0, 3, 0, 1, 1],
       [2, 2, 0, 3, 0, 0, 1, 0, 1, 1, 3, 3, 0, 2, 2, 3, 2, 3, 0, 1, 3, 1,
        3, 1, 2, 2, 2, 2, 0, 1, 1, 0, 1, 0, 0, 0, 3, 2, 0, 1, 3, 3],
       [1, 2, 1, 2, 2, 1, 2, 0, 3, 1, 0, 3, 2, 3, 1, 1, 0, 2, 3, 2, 0, 1,
        3, 3, 1, 0, 1, 0, 0, 3, 2, 0, 3, 0, 3, 0, 0, 1, 0, 2, 2, 3],
       [1, 3, 1, 1, 3, 2, 0, 3, 3, 1, 0, 3, 3, 1, 2, 2, 0, 1, 3, 0, 0, 2,
        2, 0, 3, 0, 1, 0, 3, 2, 2, 3, 0, 1, 1, 2, 0, 2, 1, 2, 0, 0]]

In [8]:
# 统计每列1, 2, 3的个数
counts = np.array([[np.sum(matrix[:,j] == i) for i in range(1, 4)] for j in range(matrix.shape[1])])

rows_with_311_counts = np.any((counts == 3) , axis=1).sum()
rows_with_320_counts  = np.any((counts == 0) , axis=1).sum()
#输出结果
print("221：{0}  311：{1}  320：{2}".format((42-rows_with_311_counts-rows_with_320_counts),rows_with_311_counts,rows_with_320_counts))

221：21  311：19  320：2


In [9]:
def get_worker_number(start, end):
    return [f'B{num:03}' for num in range(start, end + 1)]
    
worker_number = get_worker_number(1, 42)

# 创建一个空的7行21列的DataFrame，包括日期列
worker_excel = pd.DataFrame(matrix, columns=worker_number)
worker_excel.insert(0,'日期',range(1,8))
worker_excel

,日期,B001,B002,B003,B004,B005,B006,B007,B008,B009,...,B033,B034,B035,B036,B037,B038,B039,B040,B041,B042
0,1,2,3,2,3,0,2,0,2,1,...,0,3,0,1,2,1,1,0,2,2
1,2,0,0,3,0,2,3,2,1,2,...,2,1,3,3,1,3,3,2,2,2
2,3,0,0,1,0,1,3,2,1,0,...,3,3,2,2,3,0,2,3,0,0
3,4,3,1,0,1,2,0,3,3,0,...,2,2,1,2,1,0,3,0,1,1
4,5,2,2,0,3,0,0,1,0,1,...,1,0,0,0,3,2,0,1,3,3
5,6,1,2,1,2,2,1,2,0,3,...,3,0,3,0,0,1,0,2,2,3
6,7,1,3,1,1,3,2,0,3,3,...,0,1,1,2,0,2,1,2,0,0


In [10]:
def get_final_worker_excel(worker_excel):
    
    worker_number = get_worker_number(1, 42)
    # 定义一个替换字典
    replace_dict = {0: '休', 1: '早', 2: '中', 3: '晚'}

    # 使用map函数进行替换
    worker_excel[worker_number] = worker_excel[worker_number].replace(replace_dict)
    return worker_excel

In [ ]:
worker_excel = get_final_worker_excel(worker_excel)

,日期,B001,B002,B003,B004,B005,B006,B007,B008,B009,...,B033,B034,B035,B036,B037,B038,B039,B040,B041,B042
0,1,中,晚,中,晚,休,中,休,中,早,...,休,晚,休,早,中,早,早,休,中,中
1,2,休,休,晚,休,中,晚,中,早,中,...,中,早,晚,晚,早,晚,晚,中,中,中
2,3,休,休,早,休,早,晚,中,早,休,...,晚,晚,中,中,晚,休,中,晚,休,休
3,4,晚,早,休,早,中,休,晚,晚,休,...,中,中,早,中,早,休,晚,休,早,早
4,5,中,中,休,晚,休,休,早,休,早,...,早,休,休,休,晚,中,休,早,晚,晚
5,6,早,中,早,中,中,早,中,休,晚,...,晚,休,晚,休,休,早,休,中,中,晚
6,7,早,晚,早,早,晚,中,休,晚,晚,...,休,早,早,中,休,中,早,中,休,休


In [ ]:
# 将DataFrame保存为Excel文件
worker_excel.to_excel('result/result4-1.xlsx', index=False)

In [14]:
worker_number = get_worker_number(1, 42)
ages = []
ages.extend([1] * 8 + [2] * 9 + [3] * 4 + [4] * 8 + [5] * 9 + [6] * 4)

# 创建一个空的DataFrame，保存工号和工龄
worker_age = pd.DataFrame(columns=['编号', '工龄'])
worker_age['编号'] = worker_number
worker_age['工龄'] = ages

In [ ]:
# 获取问题三所得员工对不同生产线的产量和合格率
output = pd.read_csv('temp_data/员工能力数据/pivot_table_output.csv',encoding='gbk')
rate = pd.read_csv('temp_data/员工能力数据/pivot_table_rate.csv',encoding='gbk')

In [ ]:
# 量化不同工龄能力
output_np = np.array(output.iloc[:,1:])
rate_np = np.array(rate.iloc[:,1:])/100
worker_ability_info = output_np * rate_np

worker_ability = np.zeros((42,10))
for i in range(42):
    age = worker_age['工龄'][i]
    worker_ability[i,:] = worker_ability_info[age-1]

# 线性规划（人员和生产线搭配）

In [332]:
import pulp

# 线性规划寻找最优人员安排
def maximize_production(abilities):
    # 人数和生产线数
    n = 10
    
    # 创建一个线性规划问题，我们希望最大化目标函数
    prob = pulp.LpProblem("Maximize Production", pulp.LpMaximize)
    
    # 决策变量：x[i][j] 表示第 i 个人是否分配到第 j 条生产线，是则为1，否则为0
    x = pulp.LpVariable.dicts("Worker_Assignment", (range(n), range(n)), cat='Binary')
    
    # 目标函数：最大化总产量
    prob += pulp.lpSum(abilities[i][j] * x[i][j] for i in range(n) for j in range(n))
    
    # 约束条件：每个人只能分配到一条生产线
    for i in range(n):
        prob += pulp.lpSum(x[i][j] for j in range(n)) == 1
    
    # 约束条件：每条生产线只能分配1个人
    for j in range(n):
        prob += pulp.lpSum(x[i][j] for i in range(n)) == 1
    
    # 求解线性规划问题
    prob.solve()
    
    # 输出结果
    print("Maximum sorce", pulp.value(prob.objective))
    
    # 创建一个10行10列的全0矩阵
    allocation = np.zeros((10, 10), dtype=int)

    # 遍历外层字典的每个项
    for i, inner_dict in x.items():
        # 遍历内层字典的每个项
        for j, variable in inner_dict.items():
            # 检查内层字典的变量的值是否为1
            if variable.value() == 1:
                # 如果是，则将i和j作为键添加到allocation字典中
                allocation[i][j] = 1
            
    return allocation

In [334]:
def maximize_schedule(matrix, worker_ability):
    final_results = []

    for i, row in enumerate(matrix):
        for j in range(1, 4):
            # 第 i 天 第 j 班 人员安排
            indices = np.where(row == j)[0]  # 找出当前班需要工作的人员索引
            now_worker_ability = worker_ability[indices, :]  # 获取这部分工人的能力

            print("第{0}天 第{1} 时间段".format(i + 1, j))
            allocation = maximize_production(now_worker_ability)  # 进行人员分配以最大化生产

            # 根据分配结果排序
            sort = np.where(allocation == 1)[1]
            final_result = [indices[k] for k in sort]
            print(final_result)

            # 保存所有记录
            final_results.append(final_result)

    return final_results

final_results = maximize_schedule(matrix, worker_ability)

第1天 第1 时间段
Maximum sorce 94.0
[35, 37, 38, 8, 27, 31, 17, 28, 16, 20]
第1天 第2 时间段
Maximum sorce 94.0
[18, 2, 41, 7, 10, 5, 40, 36, 0, 23]
第1天 第3 时间段
Maximum sorce 91.0
[15, 12, 1, 30, 29, 3, 21, 19, 33, 25]
第2天 第1 时间段
Maximum sorce 94.0
[20, 25, 18, 7, 10, 33, 36, 21, 27, 23]
第2天 第2 时间段
Maximum sorce 88.0
[6, 19, 8, 40, 39, 32, 4, 28, 16, 41]
第2天 第3 时间段
Maximum sorce 96.0
[15, 38, 2, 22, 34, 12, 37, 31, 5, 35]
第3天 第1 时间段
Maximum sorce 90.0
[4, 2, 12, 19, 27, 25, 31, 28, 22, 7]
第3天 第2 时间段
Maximum sorce 91.0
[14, 6, 23, 9, 13, 10, 38, 34, 35, 11]
第3天 第3 时间段
Maximum sorce 92.0
[26, 36, 18, 39, 20, 24, 5, 32, 33, 16]
第4天 第1 时间段
Maximum sorce 94.0
[40, 13, 30, 1, 17, 41, 3, 36, 16, 34]
第4天 第2 时间段
Maximum sorce 91.0
[24, 28, 10, 35, 20, 11, 25, 4, 32, 33]
第4天 第3 时间段
Maximum sorce 97.0
[26, 14, 9, 6, 27, 31, 0, 38, 7, 29]
第5天 第1 时间段
Maximum sorce 90.0
[23, 9, 8, 19, 29, 32, 21, 39, 6, 30]
第5天 第2 时间段
Maximum sorce 94.0
[24, 26, 25, 13, 0, 14, 37, 27, 16, 1]
第5天 第3 时间段
Maximum sorce 95.0
[11, 22

In [ ]:
# 定义日期列表和班次列表
day = [1, 2, 3, 4, 5, 6, 7]
shifts = ['早', '中', '晚']

# 使用列表推导式生成所有日期和班次的组合，并创建一个DataFrame
times = pd.DataFrame([[d,s] for d in day for s in shifts], columns=['日期', '班次'])

# 人员搭配
final_results_B = [[f"B{num+1:03d}" for num in final_result] for final_result in final_results]
product_schedule = pd.DataFrame(final_results_B, columns=[f'M{num:03}' for num in range(1,11)])

# 最终安排
product_schedule = pd.concat((times,product_schedule),axis=1)
# 将DataFrame保存为Excel文件
product_schedule.to_excel('result/result4-2.xlsx', index=False)


,日期,班次,M001,M002,M003,M004,M005,M006,M007,M008,M009,M010
0,1,早,B036,B038,B039,B009,B028,B032,B018,B029,B017,B021
1,1,中,B019,B003,B042,B008,B011,B006,B041,B037,B001,B024
2,1,晚,B016,B013,B002,B031,B030,B004,B022,B020,B034,B026
3,2,早,B021,B026,B019,B008,B011,B034,B037,B022,B028,B024
4,2,中,B007,B020,B009,B041,B040,B033,B005,B029,B017,B042
5,2,晚,B016,B039,B003,B023,B035,B013,B038,B032,B006,B036
6,3,早,B005,B003,B013,B020,B028,B026,B032,B029,B023,B008
7,3,中,B015,B007,B024,B010,B014,B011,B039,B035,B036,B012
8,3,晚,B027,B037,B019,B040,B021,B025,B006,B033,B034,B017
9,4,早,B041,B014,B031,B002,B018,B042,B004,B037,B017,B035
